### API Gateway URL

In [6]:
api = 'https://11cgm5bwf6.execute-api.us-east-1.amazonaws.com'

### Call GET on Lambda Function

In [7]:
import requests

url = api + '/getInfo'
response = requests.get(url)
print(f'{response.status_code=}')
print(f'{response.text=}')


response.status_code=200
response.text='"Get info called!"'


### Post Image to Lambda Function - Post Image


IMG_{YYYMMDD_HHMMSS}.png

In [8]:
import requests
import base64

url = api + '/upload'
file_path = './bkr.png'

with open(file_path, 'rb') as image_file:
    response = requests.post(url, data=image_file)

if response.status_code == 200:
    print("Image uploaded successfully!")
else:
    print("Error uploading image:", response.status_code)

print(f'{response.status_code=}')
print(f'{response.text=}')

Image uploaded successfully!
response.status_code=200
response.text='"Post 1 called!"'


### Post Image to Lambda Function - Post JSON document containing Serialized Image 

In [10]:
import requests
import base64
import json
import base64
import json

def serialize_image(file_path):
    with open(file_path, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8') 
    data = {'image': encoded_image}
    json_string = json.dumps(data)
    return json_string

url = api + '/upload2'
file_path = "./bkr.png"

data = serialize_image(file_path)
response = requests.post(url, data=data)

if response.status_code == 200:
    print("Image uploaded successfully!")
else:
    print("Error uploading image:", response.status_code)

print(f'{response.status_code=}')
print(f'{response.text=}')

Image uploaded successfully!
response.status_code=200
response.text='"Post 2 called!"'
